# House Price Analytics

## 02 Data Tranformation and Loading

**Project:** Code Institute – Capstone Project

---
### **Objectives**
- Load the cleaned house dataset
- Set datatypes
- Encode categorical variables for analysis.
- Introduce required features for analaysis
- Prepare the dataset for analysis and prediction

### **Inputs**
- `data/processed/cleaned_house_data.csv`

### **Outputs**
- `data/processed/final_house_data.csv`
        
### **Additional Comments**
Confirm the cleaned_house_data.csv is exisit under outputs/datasets. Run this notebook top-down.

---

### Setup the file and Load the Dataset

Ignore warning

In [1]:
import warnings 
# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

Import nesessary libraries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

Set the home directory. Need to change the working directory from its current folder to its parent folder. Access the current directory with os.getcwd()

In [3]:
PROJECT_DIR = os.path.join(os.getcwd()) # Define the project root directory
os.chdir(PROJECT_DIR) # Change the current working directory
print("Working directory:", os.getcwd()) 

Working directory: c:\MyStuff\CodeInstitute\Projects\ci-project-02\Home-Value-Analysis\notebooks


Load the data from the original data set reside within data directory under data/processed/ directory. 

In [ ]:
# LOAD DATASET
try:
    # Data directory paths
    data_path = os.path.join("..","data", "processed")
    # Extract the original dataset
    df = pd.read_csv(os.path.join(data_path, "cleaned_house_data.csv"))
    print("Dataset loaded successfully.")
except Exception as e:
    print(e)
    print("Error loading the dataset.")
    df = pd.DataFrame()  # Create an empty DataFrame if loading fails

print(f"Original dataset shape: {df.shape}")

Dataset loaded successfully.
Original dataset shape: (21596, 21)


---

### Feature Engineering

1.  **Date Parsing:** The raw `date` string is not usable for seasonality analysis. We need to extract Year, Month, and Quarter.
2.  **Age Calculation:** Raw `yr_built` is hard to interpret. `house_age` gives immediate context.
3.  **Renovation Logic:** `yr_renovated` contains '0' for non-renovated homes. We need a binary flag `is_renovated` to compare groups.
4.  **Log Transformation:** As identified in Notebook 01, `price` is highly skewed. `price_log` normalizes this for future Machine Learning models.
5.  **Price per sqft:** Analyzing total price alone is biased by size. price_per_sqft provides a standardized metric, allowing us to compare the true "value density" of homes across different neighborhoods independent of their square footage. 

In [ ]:
# 1. Date Transformations
df['date'] = pd.to_datetime(df['date'])
df['sale_year'] = df['date'].dt.year
df['sale_month'] = df['date'].dt.month
df['sale_quarter'] = df['date'].dt.quarter

# Map month numbers to names for clearer visuals later
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
             7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
df['sale_month_name'] = df['sale_month'].map(month_map)

# 2. Age & Renovation Features
# Age at time of sale
df['house_age'] = df['sale_year'] - df['yr_built']

# Binary Renovation Flag (1 = Renovated, 0 = Not)
df['is_renovated'] = df['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)

# "Effective Age": Years since the last major update (Construction or Renovation)
df['years_since_update'] = df.apply(lambda row: row['sale_year'] - max(row['yr_built'], row['yr_renovated']), axis=1)

# 3. Price Transformations (Handling Skewness)
# Log transformation using log1p (log(1+x)) to handle any edge cases, though price > 0
df['price_log'] = np.log1p(df['price'])


# 4. Diagnostic Helper: Age Groups
# This is needed for the Seller Diagnostic Analysis in Notebook 04
bins = [1900, 1950, 1990, 2016]
labels = ['Pre-1950', '1950-1990', 'Post-1990']
df['age_group'] = pd.cut(df['yr_built'], bins=bins, labels=labels, right=False)

# 5. Price per Square Foot
# Calculated as price divided by living area in square feet
df['price_per_sqft'] = df['price'] / df['sqft_living']

# Validate transformations
print(df[['date', 'sale_month', 'house_age', 'is_renovated', 'price_log', 'price_per_sqft', 'age_group']].head())